In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
from src.models.EMOS_global.EMOS_global_load_score import *
from src.models.EMOS_local.EMOS_local_load_score import *
from src.models.EMOS_local.EMOS_local_load_model import *
from src.models.EMOS_global.EMOS_global_load_model import *
from src.models.EMOS_local.EMOS_local_hyper_tune import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *
from src.models.EMOS_local.EMOS_local_train import *

2023-06-05 12:59:05.944468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-05 12:59:05.944488: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 0. Testing until Global Hyperparameter tuning works:

In [3]:
from src.models.EMOS_global.EMOS_global_hyper_tune import *

<IPython.core.display.Javascript object>

In [4]:
var_names = ["u10", "v10", "t2m", "t850", "z500", "ws10"]
var_num = 2
lead_time = 0
epochs = [10]
batch_sizes = [1024]
lrs = [0.1]
optimizers = ["Adam"]
best_params, best_score = EMOS_global_hyper_tune(
    var_num,
    lead_time,
    batch_sizes=batch_sizes,
    epochs=epochs,
    lrs=lrs,
    optimizers=optimizers,
)
best_parms_score = [best_params, best_score]
print(best_parms_score)

  0%|                                                                                | 0/1 [00:00<?, ?it/s]2023-06-05 12:59:13.354258: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-05 12:59:13.354419: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-05 12:59:13.354548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-06-05 12:59:13.354672: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [21]:
path = f"/Data/Delong_BA_Data/scores/EMOS_global_hyper_scores/EMOS_global_hyper_{var_names[var_num]}_{lead_time}_{best_score}.pkl"
with open(path, "wb") as file:
    pickle.dump(best_parms_score, file)

<IPython.core.display.Javascript object>

### 0. Set up

In [5]:
def print_results(EMOS_global_t2m_hyper_scores, lead_num):
    print(
        f"Best params {EMOS_global_t2m_hyper_scores[lead_num][0]} score: {EMOS_global_t2m_hyper_scores[lead_num][1]} ,in lead_time {EMOS_global_t2m_hyper_scores[lead_num][2]}"
    )

    for i in range(len(EMOS_global_t2m_hyper_scores[lead_num][3])):
        print(
            EMOS_global_t2m_hyper_scores[lead_num][3][i],
            EMOS_global_t2m_hyper_scores[lead_num][4][i],
        )

<IPython.core.display.Javascript object>

### 1. Load Global Hyperparameter Scores

In [7]:
EMOS_global_t2m_hyper_scores = EMOS_global_load_hyper_score("t2m")
EMOS_global_ws10_hyper_scores = EMOS_global_load_hyper_score("ws10")

<IPython.core.display.Javascript object>

In [8]:
print(
    f"T2m-length:{len(EMOS_global_t2m_hyper_scores)}, ws10-length:{len(EMOS_global_ws10_hyper_scores)}"
)

T2m-length:3, ws10-length:3


<IPython.core.display.Javascript object>

##### Temperature

In [9]:
print_results(EMOS_global_t2m_hyper_scores, 0)

Best params (4096, 10, 0.001, 'Adam') score: 0.45013427734375 ,in lead_time 0
(1024, 10, 0.1, 'Adam') 0.4893634617328644
(1024, 10, 0.05, 'Adam') 0.45485055446624756
(1024, 10, 0.01, 'Adam') 0.4507754147052765
(1024, 10, 0.005, 'Adam') 0.45106062293052673
(1024, 10, 0.001, 'Adam') 0.4501516819000244
(2048, 10, 0.1, 'Adam') 0.47946423292160034
(2048, 10, 0.05, 'Adam') 0.4528166353702545
(2048, 10, 0.01, 'Adam') 0.4508745074272156
(2048, 10, 0.005, 'Adam') 0.450177401304245
(2048, 10, 0.001, 'Adam') 0.4502573013305664
(4096, 10, 0.1, 'Adam') 0.4852834939956665
(4096, 10, 0.05, 'Adam') 0.45073962211608887
(4096, 10, 0.01, 'Adam') 0.45114263892173767
(4096, 10, 0.005, 'Adam') 0.4501672089099884
(4096, 10, 0.001, 'Adam') 0.45013427734375
(8192, 10, 0.1, 'Adam') 0.5087556838989258
(8192, 10, 0.05, 'Adam') 0.45030805468559265
(8192, 10, 0.01, 'Adam') 0.4503491222858429
(8192, 10, 0.005, 'Adam') 0.45018625259399414
(8192, 10, 0.001, 'Adam') 0.45037367939949036


<IPython.core.display.Javascript object>

In [10]:
print_results(EMOS_global_t2m_hyper_scores, 1)

Best params (8192, 10, 0.001, 'Adam') score: 0.7493733167648315 ,in lead_time 15
(1024, 10, 0.1, 'Adam') 0.7904638648033142
(1024, 10, 0.05, 'Adam') 0.7540763020515442
(1024, 10, 0.01, 'Adam') 0.7501434683799744
(1024, 10, 0.005, 'Adam') 0.7505483031272888
(1024, 10, 0.001, 'Adam') 0.7497263550758362
(2048, 10, 0.1, 'Adam') 0.7617718577384949
(2048, 10, 0.05, 'Adam') 0.7501409649848938
(2048, 10, 0.01, 'Adam') 0.7496245503425598
(2048, 10, 0.005, 'Adam') 0.7502866387367249
(2048, 10, 0.001, 'Adam') 0.7494620084762573
(4096, 10, 0.1, 'Adam') 0.7718427181243896
(4096, 10, 0.05, 'Adam') 0.7501612901687622
(4096, 10, 0.01, 'Adam') 0.749976396560669
(4096, 10, 0.005, 'Adam') 0.7494763135910034
(4096, 10, 0.001, 'Adam') 0.7495089173316956
(8192, 10, 0.1, 'Adam') 0.7810683846473694
(8192, 10, 0.05, 'Adam') 0.7517670392990112
(8192, 10, 0.01, 'Adam') 0.7497243285179138
(8192, 10, 0.005, 'Adam') 0.7496926784515381
(8192, 10, 0.001, 'Adam') 0.7493733167648315


<IPython.core.display.Javascript object>

In [11]:
print_results(EMOS_global_t2m_hyper_scores, 2)

Best params (2048, 10, 0.01, 'Adam') score: 1.14043390750885 ,in lead_time 30
(1024, 10, 0.1, 'Adam') 1.1565638780593872
(1024, 10, 0.05, 'Adam') 1.1418256759643555
(1024, 10, 0.01, 'Adam') 1.1426897048950195
(1024, 10, 0.005, 'Adam') 1.1411614418029785
(1024, 10, 0.001, 'Adam') 1.1406571865081787
(2048, 10, 0.1, 'Adam') 1.1456468105316162
(2048, 10, 0.05, 'Adam') 1.141500473022461
(2048, 10, 0.01, 'Adam') 1.14043390750885
(2048, 10, 0.005, 'Adam') 1.1416186094284058
(2048, 10, 0.001, 'Adam') 1.141808032989502
(4096, 10, 0.1, 'Adam') 1.1449265480041504
(4096, 10, 0.05, 'Adam') 1.1533310413360596
(4096, 10, 0.01, 'Adam') 1.1412535905838013
(4096, 10, 0.005, 'Adam') 1.1413679122924805
(4096, 10, 0.001, 'Adam') 1.1412477493286133
(8192, 10, 0.1, 'Adam') 1.1749733686447144
(8192, 10, 0.05, 'Adam') 1.1452232599258423
(8192, 10, 0.01, 'Adam') 1.14060640335083
(8192, 10, 0.005, 'Adam') 1.1419819593429565
(8192, 10, 0.001, 'Adam') 1.1423572301864624


<IPython.core.display.Javascript object>

##### Wind speed

In [12]:
print_results(EMOS_global_ws10_hyper_scores, 0)

Best params (2048, 10, 0.1, 'SGD') score: 0.6247866153717041 ,in lead_time 0
(1024, 10, 0.1, 'Adam') 0.6248735785484314
(1024, 10, 0.1, 'SGD') 0.6248950958251953
(1024, 10, 0.05, 'Adam') 0.6250014901161194
(1024, 10, 0.05, 'SGD') 0.6278693079948425
(1024, 10, 0.01, 'Adam') 0.626532793045044
(1024, 10, 0.01, 'SGD') 0.6269963383674622
(1024, 10, 0.005, 'Adam') 0.6275923848152161
(1024, 10, 0.005, 'SGD') 0.6273010969161987
(1024, 10, 0.001, 'Adam') 0.6270033717155457
(1024, 10, 0.001, 'SGD') 0.6279010772705078
(2048, 10, 0.1, 'Adam') 0.6256857514381409
(2048, 10, 0.1, 'SGD') 0.6247866153717041
(2048, 10, 0.05, 'Adam') 0.6279352903366089
(2048, 10, 0.05, 'SGD') 0.6269434094429016
(2048, 10, 0.01, 'Adam') 0.6261811852455139
(2048, 10, 0.01, 'SGD') 0.6272349953651428
(2048, 10, 0.005, 'Adam') 0.6268596053123474
(2048, 10, 0.005, 'SGD') 0.6277083158493042
(2048, 10, 0.001, 'Adam') 0.6272319555282593
(2048, 10, 0.001, 'SGD') 0.6278863549232483
(4096, 10, 0.1, 'Adam') 0.6264699101448059
(4096, 

<IPython.core.display.Javascript object>

In [13]:
print_results(EMOS_global_ws10_hyper_scores, 1)

Best params (1024, 10, 0.005, 'Adam') score: 1.1340234279632568 ,in lead_time 15
(1024, 10, 0.1, 'Adam') 1.1340389251708984
(1024, 10, 0.1, 'SGD') 1.1340711116790771
(1024, 10, 0.05, 'Adam') 1.1341966390609741
(1024, 10, 0.05, 'SGD') 1.134352684020996
(1024, 10, 0.01, 'Adam') 1.1340503692626953
(1024, 10, 0.01, 'SGD') 1.1342467069625854
(1024, 10, 0.005, 'Adam') 1.1340234279632568
(1024, 10, 0.005, 'SGD') 1.1342626810073853
(1024, 10, 0.001, 'Adam') 1.13417649269104
(1024, 10, 0.001, 'SGD') 1.1343666315078735
(2048, 10, 0.1, 'Adam') 1.1349729299545288
(2048, 10, 0.1, 'SGD') 1.1340763568878174
(2048, 10, 0.05, 'Adam') 1.135208010673523
(2048, 10, 0.05, 'SGD') 1.1342941522598267
(2048, 10, 0.01, 'Adam') 1.1343153715133667
(2048, 10, 0.01, 'SGD') 1.1343286037445068
(2048, 10, 0.005, 'Adam') 1.1343283653259277
(2048, 10, 0.005, 'SGD') 1.1342934370040894
(2048, 10, 0.001, 'Adam') 1.1342017650604248
(2048, 10, 0.001, 'SGD') 1.1343393325805664
(4096, 10, 0.1, 'Adam') 1.134240984916687
(4096, 

<IPython.core.display.Javascript object>

In [14]:
print_results(EMOS_global_ws10_hyper_scores, 2)

Best params (4096, 10, 0.05, 'Adam') score: 1.4301142692565918 ,in lead_time 30
(1024, 10, 0.1, 'Adam') 1.431943655014038
(1024, 10, 0.1, 'SGD') 1.430385708808899
(1024, 10, 0.05, 'Adam') 1.4301594495773315
(1024, 10, 0.05, 'SGD') 1.4302326440811157
(1024, 10, 0.01, 'Adam') 1.430169939994812
(1024, 10, 0.01, 'SGD') 1.4304792881011963
(1024, 10, 0.005, 'Adam') 1.4301588535308838
(1024, 10, 0.005, 'SGD') 1.4307491779327393
(1024, 10, 0.001, 'Adam') 1.4306968450546265
(1024, 10, 0.001, 'SGD') 1.4308412075042725
(2048, 10, 0.1, 'Adam') 1.4304776191711426
(2048, 10, 0.1, 'SGD') 1.43023681640625
(2048, 10, 0.05, 'Adam') 1.4301786422729492
(2048, 10, 0.05, 'SGD') 1.430361032485962
(2048, 10, 0.01, 'Adam') 1.4303494691848755
(2048, 10, 0.01, 'SGD') 1.4306875467300415
(2048, 10, 0.005, 'Adam') 1.430583119392395
(2048, 10, 0.005, 'SGD') 1.4307289123535156
(2048, 10, 0.001, 'Adam') 1.4303778409957886
(2048, 10, 0.001, 'SGD') 1.4308301210403442
(4096, 10, 0.1, 'Adam') 1.43047034740448
(4096, 10, 0

<IPython.core.display.Javascript object>

### 2. Load Local Hyperparameter Scores

In [20]:
EMOS_local_t2m_hyper_scores = EMOS_local_load_hyper_score("t2m")
EMOS_local_ws10_hyper_scores = EMOS_local_load_hyper_score("ws10")

<IPython.core.display.Javascript object>

In [21]:
print(
    f"T2m-length:{len(EMOS_local_t2m_hyper_scores)}, ws10-length:{len(EMOS_local_ws10_hyper_scores)}"
)

T2m-length:3, ws10-length:3


<IPython.core.display.Javascript object>

##### Temperature:

In [22]:
print_results(EMOS_local_t2m_hyper_scores, 0)

Best params (16, 30, 0.005, 'Adam') score: 0.37988878824771977 ,in lead_time 0
(16, 30, 0.1, 'Adam') 0.49627375191985035
(16, 30, 0.05, 'Adam') 0.4216254285703867
(16, 30, 0.01, 'Adam') 0.3831466698111632
(16, 30, 0.005, 'Adam') 0.37988878824771977
(16, 30, 0.001, 'Adam') 17.344419322525845
(32, 30, 0.1, 'Adam') 0.45766198663757396
(32, 30, 0.05, 'Adam') 0.40029029032358754
(32, 30, 0.01, 'Adam') 0.38057279271575123
(32, 30, 0.005, 'Adam') 0.47604186594104153
(32, 30, 0.001, 'Adam') 114.26232341648294
(64, 30, 0.1, 'Adam') 0.5142447502376177
(64, 30, 0.05, 'Adam') 0.44653667777012557
(64, 30, 0.01, 'Adam') 0.5151060394560679
(64, 30, 0.005, 'Adam') 0.8507009725540112
(64, 30, 0.001, 'Adam') 216.2162034266079
(128, 30, 0.1, 'Adam') 0.9618315050044121
(128, 30, 0.05, 'Adam') 0.7048312280422602
(128, 30, 0.01, 'Adam') 1.0980231383672128
(128, 30, 0.005, 'Adam') 74.58889908248034
(128, 30, 0.001, 'Adam') 249.4008049772909
(256, 30, 0.1, 'Adam') 2.834916726900981
(256, 30, 0.05, 'Adam') 1.4

<IPython.core.display.Javascript object>

In [23]:
print_results(EMOS_local_t2m_hyper_scores, 1)

Best params (16, 30, 0.005, 'Adam') score: 0.7063188764911431 ,in lead_time 15
(16, 30, 0.1, 'Adam') 0.781379029345818
(16, 30, 0.05, 'Adam') 0.7365562399992576
(16, 30, 0.01, 'Adam') 0.7081086847644585
(16, 30, 0.005, 'Adam') 0.7063188764911431
(16, 30, 0.001, 'Adam') 17.594197237911896
(32, 30, 0.1, 'Adam') 0.7446841379770865
(32, 30, 0.05, 'Adam') 0.7179414452268527
(32, 30, 0.01, 'Adam') 0.706347644329071
(32, 30, 0.005, 'Adam') 0.7359531710927303
(32, 30, 0.001, 'Adam') 128.0106144963931
(64, 30, 0.1, 'Adam') 0.7903085488539475
(64, 30, 0.05, 'Adam') 0.7300917301804591
(64, 30, 0.01, 'Adam') 0.7576043812128214
(64, 30, 0.005, 'Adam') 0.8920006274412839
(64, 30, 0.001, 'Adam') 184.0715512767052
(128, 30, 0.1, 'Adam') 1.3530094715265126
(128, 30, 0.05, 'Adam') 0.8974096291722395
(128, 30, 0.01, 'Adam') 1.0043515845751152
(128, 30, 0.005, 'Adam') 87.53295544878794
(128, 30, 0.001, 'Adam') 267.54757876178394
(256, 30, 0.1, 'Adam') 3.2549329422987423
(256, 30, 0.05, 'Adam') 1.704604500

<IPython.core.display.Javascript object>

In [24]:
print_results(EMOS_local_t2m_hyper_scores, 2)

Best params (16, 30, 0.005, 'Adam') score: 1.0838735229693925 ,in lead_time 30
(16, 30, 0.1, 'Adam') 1.1478227969163504
(16, 30, 0.05, 'Adam') 1.1052566770559702
(16, 30, 0.01, 'Adam') 1.0850750077993443
(16, 30, 0.005, 'Adam') 1.0838735229693925
(16, 30, 0.001, 'Adam') 15.558259987296202
(32, 30, 0.1, 'Adam') 1.1079142334369512
(32, 30, 0.05, 'Adam') 1.090742947963568
(32, 30, 0.01, 'Adam') 1.0857606308582501
(32, 30, 0.005, 'Adam') 1.1531194135164604
(32, 30, 0.001, 'Adam') 109.61724732434138
(64, 30, 0.1, 'Adam') 1.123578464373564
(64, 30, 0.05, 'Adam') 1.0935281713803608
(64, 30, 0.01, 'Adam') 1.1697963350094283
(64, 30, 0.005, 'Adam') 1.320859425343
(64, 30, 0.001, 'Adam') 194.04333809056342
(128, 30, 0.1, 'Adam') 1.6871731460858614
(128, 30, 0.05, 'Adam') 1.407067013856692
(128, 30, 0.01, 'Adam') 1.4358300608702195
(128, 30, 0.005, 'Adam') 83.94458062373675
(128, 30, 0.001, 'Adam') 236.01420517342214
(256, 30, 0.1, 'Adam') 3.0241014468364225
(256, 30, 0.05, 'Adam') 2.124276435145

<IPython.core.display.Javascript object>

##### Wind Speed

In [25]:
print_results(EMOS_local_ws10_hyper_scores, 0)

Best params (64, 30, 0.1, 'Adam') score: 0.5987810954833642 ,in lead_time 0
(32, 30, 0.1, 'Adam') 0.5989483954050602
(32, 30, 0.1, 'SGD') 0.6794286073209383
(32, 30, 0.05, 'Adam') 0.5993755188507911
(32, 30, 0.05, 'SGD') 0.6050876546173524
(32, 30, 0.01, 'Adam') 0.6033720479179652
(32, 30, 0.01, 'SGD') 0.6048767089079587
(32, 30, 0.005, 'Adam') 0.606172231814036
(32, 30, 0.005, 'SGD') 0.6074436995654534
(32, 30, 0.001, 'Adam') 0.6225756370486357
(32, 30, 0.001, 'SGD') 0.6208223901115931
(64, 30, 0.1, 'Adam') 0.5987810954833642
(64, 30, 0.1, 'SGD') 0.7279265223978422
(64, 30, 0.05, 'Adam') 0.6001894321197119
(64, 30, 0.05, 'SGD') 0.6032571896719627
(64, 30, 0.01, 'Adam') 0.6049282004435858
(64, 30, 0.01, 'SGD') 0.6073775272338818
(64, 30, 0.005, 'Adam') 0.6102765940893919
(64, 30, 0.005, 'SGD') 0.6134857069223355
(64, 30, 0.001, 'Adam') 0.7704542474104807
(64, 30, 0.001, 'SGD') 0.633939979932247
(128, 30, 0.1, 'Adam') 0.6001739102678422
(128, 30, 0.1, 'SGD') 0.7562125863937231
(128, 30,

<IPython.core.display.Javascript object>

In [27]:
print_results(EMOS_local_ws10_hyper_scores, 1)

Best params (64, 30, 0.1, 'Adam') score: 1.1464913334601965 ,in lead_time 15
(32, 30, 0.1, 'Adam') 1.1471663965628698
(32, 30, 0.1, 'SGD') 1.1529479882656
(32, 30, 0.05, 'Adam') 1.1468491206566493
(32, 30, 0.05, 'SGD') 1.1520089321793654
(32, 30, 0.01, 'Adam') 1.148507986886379
(32, 30, 0.01, 'SGD') 1.1567316525257552
(32, 30, 0.005, 'Adam') 1.1508185116526408
(32, 30, 0.005, 'SGD') 1.1601466924334183
(32, 30, 0.001, 'Adam') 1.1677146445100124
(32, 30, 0.001, 'SGD') 1.1990550834781084
(64, 30, 0.1, 'Adam') 1.1464913334601965
(64, 30, 0.1, 'SGD') 1.1521924821994243
(64, 30, 0.05, 'Adam') 1.1466526618370643
(64, 30, 0.05, 'SGD') 1.1526299225978363
(64, 30, 0.01, 'Adam') 1.1500867475301793
(64, 30, 0.01, 'SGD') 1.1602032518921754
(64, 30, 0.005, 'Adam') 1.1535157994964185
(64, 30, 0.005, 'SGD') 1.1697707590766442
(64, 30, 0.001, 'Adam') 1.2652047228736756
(64, 30, 0.001, 'SGD') 1.2275816226998966
(128, 30, 0.1, 'Adam') 1.1477612532102144
(128, 30, 0.1, 'SGD') 1.1543301896024973
(128, 30, 

<IPython.core.display.Javascript object>

In [28]:
print_results(EMOS_local_ws10_hyper_scores, 2)

Best params (32, 30, 0.05, 'Adam') score: 1.460105578486736 ,in lead_time 30
(32, 30, 0.1, 'Adam') 1.4606935576750681
(32, 30, 0.1, 'SGD') 1.4727443806253946
(32, 30, 0.05, 'Adam') 1.460105578486736
(32, 30, 0.05, 'SGD') 1.4736094400286674
(32, 30, 0.01, 'Adam') 1.4657837135287433
(32, 30, 0.01, 'SGD') 1.4796271983247538
(32, 30, 0.005, 'Adam') 1.4703167569942963
(32, 30, 0.005, 'SGD') 1.4820695632161238
(32, 30, 0.001, 'Adam') 1.490070137840051
(32, 30, 0.001, 'SGD') 1.510183056195577
(64, 30, 0.1, 'Adam') 1.4605731188486784
(64, 30, 0.1, 'SGD') 1.4734687969470635
(64, 30, 0.05, 'Adam') 1.4618235759628124
(64, 30, 0.05, 'SGD') 1.4766312984701915
(64, 30, 0.01, 'Adam') 1.470451527681106
(64, 30, 0.01, 'SGD') 1.4826093421150477
(64, 30, 0.005, 'Adam') 1.4758542543038344
(64, 30, 0.005, 'SGD') 1.4884808756984198
(64, 30, 0.001, 'Adam') 1.5738424599552765
(64, 30, 0.001, 'SGD') 1.5468422017800503
(128, 30, 0.1, 'Adam') 1.46449460203831
(128, 30, 0.1, 'SGD') 1.4761654692582595
(128, 30, 0.

<IPython.core.display.Javascript object>

### 3. Scores for comparison:

In [29]:
# Load EMOS local scores
EMOS_local_t2m_scores = EMOS_local_load_score("t2m")
# EMOS_local_ws10_scores = EMOS_local_load_score("ws10")

# Load EMOS global scores
EMOS_global_t2m_scores = EMOS_global_load_score("t2m")
EMOS_global_ws10_scores = EMOS_global_load_score("ws10")

# Load CRPS scores
CRPS_global_t2m_scores = crps_load_lead_lat_lon("t2m")
CRPS_global_ws10_scores = crps_load_lead_lat_lon("ws10")

<IPython.core.display.Javascript object>

In [30]:
print("Temperature:")
for lead_time in range(len(EMOS_local_t2m_scores)):
    print(
        f"Lead {lead_time}:EMOS_loc: {EMOS_local_t2m_scores[lead_time].mean()}, EMOS_glob: {EMOS_global_t2m_scores[lead_time].mean()}, CRPS: {CRPS_global_t2m_scores[lead_time].mean()}"
    )

Temperature:
Lead 0:EMOS_loc: 0.4148695374260971, EMOS_glob: 0.46703651029923665, CRPS: 0.5084816639136492
Lead 1:EMOS_loc: 0.4570583921154468, EMOS_glob: 0.5298957333000098, CRPS: 0.5347707002046389
Lead 2:EMOS_loc: 0.4764913273333677, EMOS_glob: 0.5111855084322628, CRPS: 0.5150757989800734
Lead 3:EMOS_loc: 0.4853422424293378, EMOS_glob: 0.5183820239839588, CRPS: 0.5301602687833538
Lead 4:EMOS_loc: 0.5088349645341457, EMOS_glob: 0.5376392532805744, CRPS: 0.5624034501418446
Lead 5:EMOS_loc: 0.5484872642343575, EMOS_glob: 0.5939897328842008, CRPS: 0.6306282599852645
Lead 6:EMOS_loc: 0.5712123582837434, EMOS_glob: 0.5924088144510241, CRPS: 0.6316119778923832
Lead 7:EMOS_loc: 0.5754663166502546, EMOS_glob: 0.6024678559619754, CRPS: 0.6454345388357919
Lead 8:EMOS_loc: 0.6026712332169245, EMOS_glob: 0.6265648309789082, CRPS: 0.6802850949781106
Lead 9:EMOS_loc: 0.6562236252654708, EMOS_glob: 0.6937458851013558, CRPS: 0.7561511514525252
Lead 10:EMOS_loc: 0.6800776498304495, EMOS_glob: 0.69584

<IPython.core.display.Javascript object>

In [31]:
print("wind speed:")
for lead_time in range(len(EMOS_global_ws10_scores)):
    print(
        f"Lead {lead_time}: EMOS_glob: {EMOS_global_ws10_scores[lead_time].mean()}, CRPS: {CRPS_global_ws10_scores[lead_time].mean()}"
    )

wind speed:


<IPython.core.display.Javascript object>